In [ ]:
#| default_exp methods.roar

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from relax.import_essentials import *
from relax.data import TabularDataModule
from relax.utils import validate_configs
from relax.methods.vanilla import binary_cross_entropy
from relax.methods.base import BaseCFModule, BaseParametricCFModule, BasePredFnCFModule
from relax.utils import binary_cross_entropy, grad_update, cat_normalize
from rocourse_net.lime import LocalApprox
from rocourse_net.module import l_inf_proj
from sklearn.utils import gen_even_slices
from sklearn.utils.validation import _num_samples
from joblib import effective_n_jobs

In [ ]:
#| exporti
def _pred_fn_approx(
    coef: jnp.DeviceArray, 
    x: jnp.DeviceArray
) -> jnp.DeviceArray:
    return 1 / (1 + jnp.exp(- x @ coef))

In [ ]:
#| exporti
def _roar(
    x: jnp.DeviceArray, # input
    coef: jnp.DeviceArray, # lime approx weight
    intercept: jnp.DeviceArray,
    n_steps: int,
    lr: float, # learning rate for each `cf` optimization step
    lambda_: float,
    cat_arrays: List[List[str]],
    cat_idx: int,
    max_delta: float,
    n_attacker_steps: int,
    seed: int
) -> jnp.DeviceArray:
    def adv_loss(delta, coef, cf, target):
        pred = _pred_fn_approx(coef + delta, cf)
        return jnp.mean(optax.l2_loss(pred, target))

    def loss_fn_1(cf_y, y_prime):
        return jnp.mean(binary_cross_entropy(cf_y, y_prime))

    def loss_fn_2(x, cf):
        return jnp.mean(optax.l2_loss(x, cf))

    def loss_fn(cf, x, y_prime, coef):
        cf_y = _pred_fn_approx(coef, cf)
        return loss_fn_1(cf_y, y_prime) + lambda_ * loss_fn_2(x, cf)

    def adv_step(cf, coef, target, eps: float, n_steps: int):
        delta = jax.random.uniform(
            key=jax.random.PRNGKey(seed), shape=coef.shape, minval=-eps, maxval=eps)

        alpha = 1.25 * eps
        g = jax.grad(adv_loss)(delta, coef, cf, target)
        delta = delta + alpha * jnp.sign(g)

        delta = l_inf_proj(delta.reshape(1, -1), eps=eps, cat_idx=cat_idx).reshape(-1)
        return delta

    @ jax.jit
    def gen_cf_step(x, cf, opt_state: optax.OptState):
        delta = adv_step(
            cf, coef, y_target, 
            max_delta, 
            n_attacker_steps
        )

        g = jax.grad(loss_fn)(cf, x, y_target, coef + delta)
        cf, opt_state = grad_update(g, cf, opt_state, opt)
        cf = cat_normalize(cf, cat_arrays, cat_idx, hard=False)
        return cf, opt_state

    x = x.reshape(1, -1)
    cf = jnp.array(x, copy=True)
    
    y_pred = _pred_fn_approx(coef, x)
    y_target = 1. - jnp.round(y_pred)

    opt = optax.rmsprop(lr)
    opt_state = opt.init(cf)
    for _ in tqdm(range(n_steps)):
        cf, opt_state = gen_cf_step(x, cf, opt_state)

    cf = cat_normalize(
        cf, cat_arrays, cat_idx, hard=True)
    return cf.reshape(-1)

In [ ]:
#| exporti
def _generate_local_exp(
    test_X: chex.ArrayBatched,
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray],
    train_X: chex.ArrayBatched,
    cat_arrays: List[List[str]],
    cat_idx: int,
    n_jobs: int
):
    def pred_proba(x: jnp.DeviceArray):
        if len(x.shape) == 1:
            x = x.reshape(1, -1)
        prob = pred_fn(x)
        return jnp.concatenate([1-prob, prob], axis=1)

    def local_exp_step(x, coef_, intercept_, i):
        x = x.reshape(1, -1)
        coef, intercept = lime.extract_weights(x, pred_proba)
        coef_[i, :] = coef
        intercept_[i, :] = intercept
        
    lime = LocalApprox(
        train_X, cat_arrays, cat_idx
    )

    coef_, intercept_ = np.empty(test_X.shape, order='F'), np.empty(test_X.shape, order='F')

    if n_jobs == 1:
        for i, x in enumerate(tqdm(test_X)):
            local_exp_step(x, coef_, intercept_, i)
    else:
        # https://github.com/scikit-learn/scikit-learn/blob/36958fb240fbe435673a9e3c52e769f01f36bec0/sklearn/metrics/pairwise.py#L1568
        Parallel(n_jobs=n_jobs, backend='threading')(
            delayed(local_exp_step)(x, coef_, intercept_, i)
            for i, x in enumerate(tqdm(test_X))
        )
    return jnp.array(coef_), jnp.array(intercept_)
    

In [ ]:
#| export
class ROARConfig(BaseParser):
    max_delta: float = 0.1
    n_steps: int = 50
    n_attacker_steps: int = 10
    lambda_: float = 0.5
    lr: float = 0.1
    seed: int = 42

In [ ]:
#| export
class ROAR(BaseCFModule):
    name = "ROAR"

    def __init__(
        self,
        configs: Union[Dict[str, Any], ROARConfig],
        data_module: Optional[TabularDataModule] = None
    ):
        self.configs = validate_configs(configs, ROARConfig)
        if data_module:
            self.update_cat_info(data_module)

    def update_cat_info(self, data_module: TabularDataModule):
        self.X, _  = data_module.train_dataset[:]
        return super().update_cat_info(data_module)

    def generate_cf(self, 
        x: jnp.DeviceArray, # input
        coef: jnp.DeviceArray, # lime approx weight
        intercept: jnp.DeviceArray
    ) -> jnp.DeviceArray:
        return _roar(
            x=x, # input
            coef=coef,
            intercept=intercept,
            n_steps=self.configs.n_steps,
            lr=self.configs.lr, # learning rate for each `cf` optimization step
            lambda_=self.configs.lambda_,
            cat_arrays=self.cat_arrays,
            cat_idx=self.cat_idx,
            max_delta=self.configs.max_delta,
            n_attacker_steps=self.configs.n_attacker_steps,
            seed=self.configs.seed
        )

    def generate_cfs(
        self, 
        X: chex.ArrayBatched, 
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray], 
        is_parallel: bool = False
    ) -> chex.ArrayBatched:
        print('generating local explanations via lime...')
        X, _ = self.data_module.train_dataset[:]
        self.cat_arrays = self.data_module.cat_encoder.categories_ \
            if self.data_module._configs.discret_cols else []
        self.cat_idx = self.data_module.cat_idx
        coef, intercept = _generate_local_exp(
            test_X=X, pred_fn=pred_fn, train_X=X,
            cat_arrays=self.cat_arrays, 
            cat_idx=self.cat_idx, n_jobs=-1
        )

        def _generate_cf(x: jnp.DeviceArray, coef, intercept) -> jnp.ndarray:
            return self.generate_cf(x, coef, intercept)
        return jax.vmap(_generate_cf)(X, coef, intercept) if not is_parallel \
            else jax.pmap(_generate_cf)(X, coef, intercept)

In [ ]:
# deprecated
# class ROAR(BaseCFModule):
    name = "ROAR"

    def __init__(
        self,
        configs: Union[Dict[str, Any], ROARConfig],
        data_module: Optional[TabularDataModule] = None
    ):
        self.configs = validate_configs(configs, ROARConfig)
        if data_module:
            self.update_cat_info(data_module)

    def update_cat_info(self, data_module: TabularDataModule):
        self.X, _  = data_module.train_dataset[:]
        return super().update_cat_info(data_module)

    def pred_fn_approx(self, coef, x):
        return 1 / (1 + jnp.exp(- x @ coef))

    def generate_cf(self, 
        x: jnp.DeviceArray, # input
        coef: jnp.DeviceArray, # lime approx weight
        intercept: jnp.DeviceArray
    ) -> jnp.DeviceArray:
        def adv_loss(delta, coef, cf, target):
            pred = self.pred_fn_approx(coef + delta, cf)
            return jnp.mean(optax.l2_loss(pred, target))

        def loss_fn_1(cf_y, y_prime):
            # return jnp.mean(optax.l2_loss(cf_y, y_prime))
            return jnp.mean(binary_cross_entropy(cf_y, y_prime))

        def loss_fn_2(x, cf):
            return jnp.mean(optax.l2_loss(x, cf))

        def loss_fn(cf, x, y_prime, coef):
            cf_y = self.pred_fn_approx(coef, cf)
            return loss_fn_1(cf_y, y_prime) + self.configs.lambda_ * loss_fn_2(x, cf)

        def adv_step(cf, coef, target, eps: float, n_steps: int):
            delta = jax.random.uniform(
                key=next(self.configs.keys), 
                shape=coef.shape, minval=-eps, maxval=eps)

            alpha = 1.25 * eps
            g = jax.grad(adv_loss)(delta, coef, cf, target)
            delta = delta + alpha * jnp.sign(g)

            delta = l_inf_proj(delta.reshape(1, -1), eps=eps, cat_idx=self.cat_idx).reshape(-1)
            return delta

        @ jax.jit
        def gen_cf_step(x, cf, opt_state: optax.OptState):
            delta = adv_step(
                cf, coef, y_target, 
                self.configs.max_delta, 
                self.configs.n_attacker_steps
            )

            g = jax.grad(loss_fn)(cf, x, y_target, coef + delta)
            cf, opt_state = grad_update(g, cf, opt_state, opt)
            cf = cat_normalize(cf, self.cat_arrays, self.cat_idx, hard=False)
            return cf, opt_state

        x = x.reshape(1, -1)
        cf = jnp.array(x, copy=True)
        
        y_pred = self.pred_fn_approx(coef, x)
        y_target = 1. - jnp.round(y_pred)

        opt = optax.rmsprop(self.configs.lr)
        opt_state = opt.init(cf)
        for _ in tqdm(range(self.configs.n_steps)):
            cf, opt_state = gen_cf_step(x, cf, opt_state)

        cf = cat_normalize(
            cf, self.cat_arrays, self.cat_idx, hard=True)
        return cf.reshape(-1)
        
    def generate_local_exps(
        self,
        X: chex.ArrayBatched, 
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray]
    ):
        def _pred_fn(x: jnp.DeviceArray):
            if len(x.shape) == 1:
                x = x.reshape(1, -1)
            prob = pred_fn(x)
            return jnp.concatenate([1-prob, prob], axis=1)

        lime = LocalApprox(self.X, self.cat_arrays, self.cat_idx)
        coef_, intercept_ = [], []
        for x in tqdm(X): # TODO: optimize for parallel computing
            x = x.reshape(1, -1)
            coef, intercept = lime.extract_weights(x, _pred_fn)
            # assert jnp.abs(self.pred_fn_approx(coef, x) - \
            #     pred_fn(x)) < 0.05, \
            #     f"self.pred_fn_approx(coef, x)={self.pred_fn_approx(coef, x)}" + \
            #         f"pred_fn(x)={pred_fn(x)}"
            coef_.append(coef); intercept_.append(intercept)
        return np.array(coef_), np.array(intercept_)
    
    def generate_cfs(
        self, 
        X: chex.ArrayBatched, 
        pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray], 
        is_parallel: bool = False
    ) -> chex.ArrayBatched:
        print('generating local explanations via lime...')
        coef, intercept = self.generate_local_exps(X, pred_fn)
        def _generate_cf(x: jnp.DeviceArray, coef, intercept) -> jnp.ndarray:
            return self.generate_cf(x, coef, intercept)
        return jax.vmap(_generate_cf)(X, coef, intercept) if not is_parallel \
            else jax.pmap(_generate_cf)(X, coef, intercept)

Test

In [ ]:
from relax.trainer import train_model
from relax.module import PredictiveTrainingModule
from relax.evaluate import generate_cf_explanations


dm = TabularDataModule({
    "data_name": "student",
    "continous_cols": [
        "failures", "age"
    ],
    "discret_cols": [
        "G2", "G1", "higher", "goout", "Mjob", "Fjob", "health", 
        "freetime", "absences", "Walc", "famrel", "Medu", "Fedu"
    ],
    "data_dir": "assets/data/student/gp.csv"
})
m_configs = {
    'lr': 0.03,
    "sizes": [50, 10, 50],
    "dropout_rate": 0.3,
}
t_configs = {
    'n_epochs': 10,
    'monitor_metrics': 'val/val_loss',
    'logger_name': 'pred',
    "batch_size": 128,
}

dm.prepare_data()

training_module = PredictiveTrainingModule(m_configs)
param, _ = train_model(
    training_module, dm, t_configs=t_configs
)

Epoch 9: 100%|██████████| 3/3 [00:00<00:00, 100.39batch/s, train/train_loss_1=0.0374]


In [ ]:
from copy import deepcopy

_param = deepcopy(param)
pred_fn = lambda x: training_module.forward(_param, random.PRNGKey(0), x, is_training=False)

In [ ]:
# test prediction
X, y = dm.test_dataset[:]
jnp.mean(jnp.round(pred_fn(X)) == y)

DeviceArray(0.8490566, dtype=float32)

In [ ]:
?generate_cf_explanations

Signature:
generate_cf_explanations(
    cf_module: 'BaseCFModule',
    datamodule: 'TabularDataModule',
    pred_fn: 'callable' = None,
    t_configs: 'TrainingConfigs' = None,
    pred_fn_args: 'dict' = None,
) -> 'Explanation'
Docstring: Generate CF explanations.
File:      ~/code/ReLax/relax/evaluate.py
Type:      function


In [ ]:
cf_exp = ROAR({'lr': 0.01, 'n_steps': 100, 'lambda_': 0.5})
res = generate_cf_explanations(
    cf_exp, dm, pred_fn,)

generating local explanations via lime...


100%|██████████| 100/100 [00:01<00:00, 64.51it/s]
